In [1]:
import os, sys, shutil
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
sys.path.append("../src")
from utils import *
from matplotlib.lines import Line2D
import matplotlib as mpl
import math
import matplotlib.image as mpimg
import random
from transformers import AutoProcessor, AutoTokenizer, CLIPVisionModelWithProjection, CLIPTextModelWithProjection
os.chdir("../")
from reconstructor import Reconstructor
# from reconstructor_bd import Reconstructor

/export/raid1/home/kneel027/miniconda3/envs/SS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/export/raid1/home/kneel027/miniconda3/envs/SS/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/export/raid1/home/kneel027/miniconda3/envs/SS/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# R = Reconstructor(fp16=False, device="cuda:2")
R = Reconstructor(device="cuda:0")


#######################
# Running in eps mode #
#######################

making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Load pth from pretrained/kl-f8.pth
Load autoencoderkl with total 83653863 parameters,72921.759 parameter sum.
Load optimus_bert_connector with total 109489920 parameters,19108.919 parameter sum.
Load optimus_gpt2_connector with total 132109824 parameters,19220.966 parameter sum.
Load pth from pretrained/optimus-vae.pth
Load optimus_vae_next with total 241599744 parameters,-344611.688 parameter sum.


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Load clip_image_context_encoder with total 427616513 parameters,64007.510 parameter sum.
Load clip_text_context_encoder with total 427616513 parameters,64007.510 parameter sum.
Load openai_unet_2d_next with total 859520964 parameters,100402.990 parameter sum.
Load openai_unet_0d_next with total 1706797888 parameters,250096.403 parameter sum.
Load vd_v2_0 with total 3746805485 parameters,206824.483 parameter sum.
Reconstructor initialized


In [3]:
base_path = "/home/naxos2-raid25/kneel027/home/kneel027/Second-Sight/output/mental_imagery_paper/"
version = "2.3"
for mode in ["vision", "imagery"]:
    for subject in [1, 2, 5, 7]:
        for sample in tqdm(range(0, 1)):
            sample_path = base_path + f"{mode}/brain-optimized-inference-v{version}/subject{subject}/{sample}/"
            image_path = sample_path + f"best_distribution/images/"
            for rep in range(10):
                if os.path.exists(image_path + f"{rep}.png"):
                    shutil.copy(image_path + f"{rep}.png", sample_path + f"{rep}.png")
                else:
                    if version == "2.3":
                        clip_vis = torch.load(sample_path + "clip_vision.pt", map_location="cpu")
                        clip_text = torch.load(sample_path + "clip_text.pt", map_location="cpu")
                        textstrength=0.4
                    elif version == "2.1":
                        clip_vis = torch.load(sample_path + "clip_best.pt", map_location="cpu")
                        clip_text = None
                        textstrength=0.0
                    z_img = Image.open(sample_path + "best_distribution/z_img.png")
                    with open(sample_path + "best_distribution/strength.txt", "r") as file:
                        strength = float(file.read())
                    new_img = R.reconstruct(image=z_img,
                                            c_i=clip_vis,
                                            c_t=clip_text,
                                            strength=0.99,
                                            textstrength=textstrength)
                    new_img.save(sample_path + f"{rep}.png")
                    new_img.save(image_path + f"{rep}.png")
                    # print(sample_path + f"{rep}.png")
                    


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 259.82it/s]
